# Pynq GPIO tutorial using MMIO


Load Tutorial Overlay
![diagram](data/diagram.png)

In [1]:
import pynq
class TutorialOverlay(pynq.Overlay):
    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)

ol = TutorialOverlay("/home/xilinx/jupyter_notebooks/pynq_tutorial/hw/bitstream/pynq_tutorial.bit")

Check addresses of IP in the overlay
![address_editor](data/address_editor.png)

In [2]:
import pprint as pp
pp.pprint(ol.ip_dict)

{'axi_dma_from_pl_to_ps': {'addr_range': 65536,
                           'driver': <class 'pynq.lib.dma.DMA'>,
                           'fullpath': 'axi_dma_from_pl_to_ps',
                           'gpio': {},
                           'interrupts': {'s2mm_introut': {'controller': 'system_interrupts',
                                                           'fullpath': 'axi_dma_from_pl_to_ps/s2mm_introut',
                                                           'index': 1}},
                           'phys_addr': 1078001664,
                           'state': None,
                           'type': 'xilinx.com:ip:axi_dma:7.1'},
 'axi_dma_from_ps_to_pl': {'addr_range': 65536,
                           'driver': <class 'pynq.lib.dma.DMA'>,
                           'fullpath': 'axi_dma_from_ps_to_pl',
                           'gpio': {},
                           'interrupts': {'mm2s_introut': {'controller': 'system_interrupts',
                                       

The addresses are in decimal. Check the address of the LED controller:

In [3]:
led_addr = ol.ip_dict["swsleds_gpio"]['phys_addr']
print("0x" + format(led_addr, '02x'))

0x41200000


## Set up PL GPIO LED controller

Two LEDs are connected to PL and will be accessed from the MMIO class. Note the LEDs are on channel 2 of the GPIO controller.

In [4]:
from pynq import MMIO
CH2_OFFSET = 0x8
leds = MMIO(led_addr + CH2_OFFSET)

## Write to the LEDs
This shoudl write to LD1 and LD0

In [5]:
leds.write(0, 1) # Turn on LD0

In [6]:
leds.write(0, 2) # Turn on LD1, and LD0 off

In [7]:
leds.write(0, 0) # Turn off both LEDs

## Set up PL GPIO buttons controller

Two push-buttons are connected to PL and will be accessed from the MMIO class. 

In [8]:
from pynq import MMIO
btn_addr = ol.ip_dict["btns_gpio"]["phys_addr"]
buttons = MMIO(btn_addr)

## Read from pushbuttons

This will read a 2-bit value from BTN1 and BTN0

In [9]:
buttons.read()

0

## Set up PS EMIO GPIO LED controller

In [10]:
from pynq import GPIO

EMIO GPIO pins 7 and 8 are connected to LD3 and LD4. GPIO pins need a lookup to determine the linux number of the GPIO pin. This is done with ** get_gpio_pin() **

In [11]:
ps_led2 = GPIO(GPIO.get_gpio_pin(7), 'out')
ps_led3 = GPIO(GPIO.get_gpio_pin(8), 'out')

Write something to the LEDs

In [12]:
ps_led2.write(0)
ps_led3.write(1)

In [13]:
ps_led2.write(1)
ps_led3.write(0)

In [14]:
ps_led2.write(0)
ps_led3.write(0)

## Set up PS EMIO GPIO buttons controller
BTN3 and BTN2 on the board are connected to EMIO GPIO 0 and 1

In [15]:
ps_btn0 = GPIO(GPIO.get_gpio_pin(0), 'in')
ps_btn1 = GPIO(GPIO.get_gpio_pin(1), 'in')

In [16]:
print(ps_btn0.read())
print(ps_btn1.read())

0
0
